In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from absl import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb


/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df
logging.set_verbosity(logging.ERROR)

train_df, test_df = download_and_load_datasets()

In [3]:
data_x=list(train_df.get("sentence"))
data_y=list(train_df.get("polarity"))
datate_x=list(test_df.get("sentence"))
datate_y=list(test_df.get("polarity"))

In [4]:
def delp(string): 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
    return string

In [13]:
import string
pr_train=[]
pr_test=[]
for x in data_x:
    x.replace("<br /><br />","")
    x.replace(r"\n", " ")
    pr_train.append(delp(x))
for x in datate_x:
    x.replace("<br /><br />","")
    x.replace(r"\n", "")
    pr_test.append(delp(x))
sentence = pr_train + pr_test
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(sentence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_x = pad_sequences(tokenizer.texts_to_sequences(pr_train), maxlen=80)
test_x = pad_sequences(tokenizer.texts_to_sequences(pr_test), maxlen=80)

In [14]:
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_x, np.array(data_y),
          batch_size=64,
          epochs=8,
          validation_data=(test_x, datate_y))

Train on 25000 samples, validate on 25000 samples
Epoch 1/8
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.4599 - accuracy: 0.7820 - val_loss: 0.3937 - val_accuracy: 0.8304
Epoch 2/8
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.3162 - accuracy: 0.8697 - val_loss: 0.3894 - val_accuracy: 0.8291
Epoch 3/8
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.2566 - accuracy: 0.8965 - val_loss: 0.5173 - val_accuracy: 0.8150
Epoch 4/8
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.2137 - accuracy: 0.9170 - val_loss: 0.4373 - val_accuracy: 0.8310
Epoch 5/8
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.1776 - accuracy: 0.9328 - val_loss: 0.4895 - val_accuracy: 0.8249
Epoch 6/8
25000/25000 [==============================] - 45s 2ms/sample - loss: 0.1394 - accuracy: 0.9491 - val_loss: 0.5483 - val_accuracy: 0.8098
Epoch 7/8
25000/25000 [==============================] - 45s 2